In [ ]:
group members = "Lucas Pedroza, Michael Willekens, Yoann Allardin, Konstantin Baron von Drachenfels"

In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
import sys
sys.path.append("/Users/bolli/fastai/old/")

from fastai.imports import *
from fastai.structured import *

from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display

from sklearn import metrics

C:\Users\bolli\Anaconda3\envs\fastai-cpu-v0.7\lib\site-packages\dask\config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
C:\Users\bolli\Anaconda3\envs\fastai-cpu-v0.7\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.ensemble.forest module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)


In [3]:
PATH = "data/cars/"

In [4]:
!ls {PATH}

training.csv


In [5]:
!head data/cars/training.csv

RefId,IsBadBuy,PurchDate,Auction,VehYear,VehicleAge,Make,Model,Trim,SubModel,Color,Transmission,WheelTypeID,WheelType,VehOdo,Nationality,Size,TopThreeAmericanName,MMRAcquisitionAuctionAveragePrice,MMRAcquisitionAuctionCleanPrice,MMRAcquisitionRetailAveragePrice,MMRAcquisitonRetailCleanPrice,MMRCurrentAuctionAveragePrice,MMRCurrentAuctionCleanPrice,MMRCurrentRetailAveragePrice,MMRCurrentRetailCleanPrice,PRIMEUNIT,AUCGUART,BYRNO,VNZIP1,VNST,VehBCost,IsOnlineSale,WarrantyCost
1,0,12/7/2009,ADESA,2006,3,MAZDA,MAZDA3,i,4D SEDAN I,RED,AUTO,1,Alloy,89046,OTHER ASIAN,MEDIUM,OTHER,8155,9829,11636,13600,7451,8552,11597,12409,NULL,NULL,21973,33619,FL,7100,0,1113
2,0,12/7/2009,ADESA,2004,5,DODGE,1500 RAM PICKUP 2WD,ST,QUAD CAB 4.7L SLT,WHITE,AUTO,1,Alloy,93593,AMERICAN,LARGE TRUCK,CHRYSLER,6854,8383,10897,12572,7456,9222,11374,12791,NULL,NULL,19638,33619,FL,7600,0,1053
3,0,12/7/2009,ADESA,2005,4,DODGE,STRATUS V6,SXT,4D SEDAN SXT FFV,MAROON,AUTO,2,Covers,73807,AMERICAN,MEDIUM,CHRYSLER,3202,4760,694

In [6]:
df_raw = pd.read_csv(f'{PATH}training.csv', low_memory=False, 
                     parse_dates=["PurchDate"])

In [7]:
df_raw

,RefId,IsBadBuy,PurchDate,Auction,VehYear,VehicleAge,Make,Model,Trim,SubModel,Color,Transmission,WheelTypeID,WheelType,VehOdo,Nationality,Size,TopThreeAmericanName,MMRAcquisitionAuctionAveragePrice,MMRAcquisitionAuctionCleanPrice,MMRAcquisitionRetailAveragePrice,MMRAcquisitonRetailCleanPrice,MMRCurrentAuctionAveragePrice,MMRCurrentAuctionCleanPrice,MMRCurrentRetailAveragePrice,MMRCurrentRetailCleanPrice,PRIMEUNIT,AUCGUART,BYRNO,VNZIP1,VNST,VehBCost,IsOnlineSale,WarrantyCost
0,1,0,2009-12-07,ADESA,2006,3,MAZDA,MAZDA3,i,4D SEDAN I,RED,AUTO,1.0,Alloy,89046,OTHER ASIAN,MEDIUM,OTHER,8155.0,9829.0,11636.0,13600.0,7451.0,8552.0,11597.0,12409.0,NaN,NaN,21973,33619,FL,7100.0,0,1113
1,2,0,2009-12-07,ADESA,2004,5,DODGE,1500 RAM PICKUP 2WD,ST,QUAD CAB 4.7L SLT,WHITE,AUTO,1.0,Alloy,93593,AMERICAN,LARGE TRUCK,CHRYSLER,6854.0,8383.0,10897.0,12572.0,7456.0,9222.0,11374.0,12791.0,NaN,NaN,19638,33619,FL,7600.0,0,1053
2,3,0,2009-12-07,ADESA,2005,4,DODGE,STRATUS V6,SXT,4D SEDAN SXT FFV,MAROON,AUTO,2.0,Covers,73807,AMERICAN,MEDIUM,CHRYSLER,3202.0,4760.0,6943.0,8457.0,4035.0,5557.0,7146.0,8702.0,NaN,NaN,19638,33619,FL,4900.0,0,1389
3,4,0,2009-12-07,ADESA,2004,5,DODGE,NEON,SXT,4D SEDAN,SILVER,AUTO,1.0,Alloy,65617,AMERICAN,COMPACT,CHRYSLER,1893.0,2675.0,4658.0,5690.0,1844.0,2646.0,4375.0,5518.0,NaN,NaN,19638,33619,FL,4100.0,0,630
4,5,0,2009-12-07,ADESA,2005,4,FORD,FOCUS,ZX3,2D COUPE ZX3,SILVER,MANUAL,2.0,Covers,69367,AMERICAN,COMPACT,FORD,3913.0,5054.0,7723.0,8707.0,3247.0,4384.0,6739.0,7911.0,NaN,NaN,19638,33619,FL,4000.0,0,1020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72978,73010,1,2009-12-02,ADESA,2001,8,MERCURY,SABLE,GS,4D SEDAN GS,BLACK,AUTO,1.0,Alloy,45234,AMERICAN,MEDIUM,FORD,1996.0,2993.0,2656.0,3732.0,2190.0,3055.0,4836.0,5937.0,NaN,NaN,18111,30212,GA,4200.0,0,993
72979,73011,0,2009-12-02,ADESA,2007,2,CHEVROLET,MALIBU 4C,LS,4D SEDAN LS,SILVER,AUTO,NaN,NaN,71759,AMERICAN,MEDIUM,GM,6418.0,7325.0,7431.0,8411.0,6785.0,8132.0,10151.0,11652.0,NaN,NaN,18881,30212,GA,6200.0,0,1038
72980,73012,0,2009-12-02,ADESA,2005,4,JEEP,GRAND CHEROKEE 2WD V,Lar,4D WAGON LAREDO,SILVER,AUTO,1.0,Alloy,88500,AMERICAN,MEDIUM SUV,CHRYSLER,8545.0,9959.0,9729.0,11256.0,8375.0,9802.0,11831.0,14402.0,NaN,NaN,18111,30212,GA,8200.0,0,1893
72981,73013,0,2009-12-02,ADESA,2006,3,CHEVROLET,IMPALA,LS,4D SEDAN LS,WHITE,AUTO,1.0,Alloy,79554,AMERICAN,LARGE,GM,6420.0,7604.0,7434.0,8712.0,6590.0,7684.0,10099.0,11228.0,NaN,NaN,18881,30212,GA,7000.0,0,1974


In [8]:
def display_all(df):
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000): 
        display(df)

In [9]:
display_all(df_raw.tail().T)

,72978,72979,72980,72981,72982
RefId,73010,73011,73012,73013,73014
IsBadBuy,1,0,0,0,0
PurchDate,2009-12-02 00:00:00,2009-12-02 00:00:00,2009-12-02 00:00:00,2009-12-02 00:00:00,2009-12-02 00:00:00
Auction,ADESA,ADESA,ADESA,ADESA,ADESA
VehYear,2001,2007,2005,2006,2006
VehicleAge,8,2,4,3,3
Make,MERCURY,CHEVROLET,JEEP,CHEVROLET,MAZDA
Model,SABLE,MALIBU 4C,GRAND CHEROKEE 2WD V,IMPALA,MAZDA6
Trim,GS,LS,Lar,LS,s
SubModel,4D SEDAN GS,4D SEDAN LS,4D WAGON LAREDO,4D SEDAN LS,4D SEDAN S


In [10]:
display_all(df_raw.describe(include='all').T)

,count,unique,top,freq,first,last,mean,std,min,25%,50%,75%,max
RefId,72983,NaN,NaN,NaN,NaT,NaT,36511.4,21077.2,1,18257.5,36514,54764.5,73014
IsBadBuy,72983,NaN,NaN,NaN,NaT,NaT,0.122988,0.328425,0,0,0,0,1
PurchDate,72983,517,2010-11-23 00:00:00,384,2009-01-05,2010-12-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Auction,72983,3,MANHEIM,41043,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
VehYear,72983,NaN,NaN,NaN,NaT,NaT,2005.34,1.73125,2001,2004,2005,2007,2010
VehicleAge,72983,NaN,NaN,NaN,NaT,NaT,4.17664,1.71221,0,3,4,5,9
Make,72983,33,CHEVROLET,17248,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Model,72983,1063,PT CRUISER,2329,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Trim,70623,134,Bas,13950,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SubModel,72975,863,4D SEDAN,15236,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
m = RandomForestRegressor(n_jobs=-1)
# The following code is supposed to fail due to string values in the input data
m.fit(df_raw.drop('IsBadBuy', axis=1), df_raw.IsBadBuy)

TypeError: invalid type promotion

In [12]:
df_raw.PurchDate

0       2009-12-07
1       2009-12-07
2       2009-12-07
3       2009-12-07
4       2009-12-07
           ...    
72978   2009-12-02
72979   2009-12-02
72980   2009-12-02
72981   2009-12-02
72982   2009-12-02
Name: PurchDate, Length: 72983, dtype: datetime64[ns]

In [15]:
??add_datepart

In [14]:
add_datepart(df_raw, 'PurchDate')
df_raw.PurchYear.head()

0    2009
1    2009
2    2009
3    2009
4    2009
Name: PurchYear, dtype: int64

In [16]:
df_raw.columns

Index(['RefId', 'IsBadBuy', 'Auction', 'VehYear', 'VehicleAge', 'Make',
       'Model', 'Trim', 'SubModel', 'Color', 'Transmission', 'WheelTypeID',
       'WheelType', 'VehOdo', 'Nationality', 'Size', 'TopThreeAmericanName',
       'MMRAcquisitionAuctionAveragePrice', 'MMRAcquisitionAuctionCleanPrice',
       'MMRAcquisitionRetailAveragePrice', 'MMRAcquisitonRetailCleanPrice',
       'MMRCurrentAuctionAveragePrice', 'MMRCurrentAuctionCleanPrice',
       'MMRCurrentRetailAveragePrice', 'MMRCurrentRetailCleanPrice',
       'PRIMEUNIT', 'AUCGUART', 'BYRNO', 'VNZIP1', 'VNST', 'VehBCost',
       'IsOnlineSale', 'WarrantyCost', 'PurchYear', 'PurchMonth', 'PurchWeek',
       'PurchDay', 'PurchDayofweek', 'PurchDayofyear', 'PurchIs_month_end',
       'PurchIs_month_start', 'PurchIs_quarter_end', 'PurchIs_quarter_start',
       'PurchIs_year_end', 'PurchIs_year_start', 'PurchElapsed'],
      dtype='object')

In [18]:
df_raw.head()

,RefId,IsBadBuy,Auction,VehYear,VehicleAge,Make,Model,Trim,SubModel,Color,Transmission,WheelTypeID,WheelType,VehOdo,Nationality,Size,TopThreeAmericanName,MMRAcquisitionAuctionAveragePrice,MMRAcquisitionAuctionCleanPrice,MMRAcquisitionRetailAveragePrice,MMRAcquisitonRetailCleanPrice,MMRCurrentAuctionAveragePrice,MMRCurrentAuctionCleanPrice,MMRCurrentRetailAveragePrice,MMRCurrentRetailCleanPrice,PRIMEUNIT,AUCGUART,BYRNO,VNZIP1,VNST,VehBCost,IsOnlineSale,WarrantyCost,PurchYear,PurchMonth,PurchWeek,PurchDay,PurchDayofweek,PurchDayofyear,PurchIs_month_end,PurchIs_month_start,PurchIs_quarter_end,PurchIs_quarter_start,PurchIs_year_end,PurchIs_year_start,PurchElapsed
0,1,0,ADESA,2006,3,MAZDA,MAZDA3,i,4D SEDAN I,RED,AUTO,1.0,Alloy,89046,OTHER ASIAN,MEDIUM,OTHER,8155.0,9829.0,11636.0,13600.0,7451.0,8552.0,11597.0,12409.0,NaN,NaN,21973,33619,FL,7100.0,0,1113,2009,12,50,7,0,341,False,False,False,False,False,False,1260144000
1,2,0,ADESA,2004,5,DODGE,1500 RAM PICKUP 2WD,ST,QUAD CAB 4.7L SLT,WHITE,AUTO,1.0,Alloy,93593,AMERICAN,LARGE TRUCK,CHRYSLER,6854.0,8383.0,10897.0,12572.0,7456.0,9222.0,11374.0,12791.0,NaN,NaN,19638,33619,FL,7600.0,0,1053,2009,12,50,7,0,341,False,False,False,False,False,False,1260144000
2,3,0,ADESA,2005,4,DODGE,STRATUS V6,SXT,4D SEDAN SXT FFV,MAROON,AUTO,2.0,Covers,73807,AMERICAN,MEDIUM,CHRYSLER,3202.0,4760.0,6943.0,8457.0,4035.0,5557.0,7146.0,8702.0,NaN,NaN,19638,33619,FL,4900.0,0,1389,2009,12,50,7,0,341,False,False,False,False,False,False,1260144000
3,4,0,ADESA,2004,5,DODGE,NEON,SXT,4D SEDAN,SILVER,AUTO,1.0,Alloy,65617,AMERICAN,COMPACT,CHRYSLER,1893.0,2675.0,4658.0,5690.0,1844.0,2646.0,4375.0,5518.0,NaN,NaN,19638,33619,FL,4100.0,0,630,2009,12,50,7,0,341,False,False,False,False,False,False,1260144000
4,5,0,ADESA,2005,4,FORD,FOCUS,ZX3,2D COUPE ZX3,SILVER,MANUAL,2.0,Covers,69367,AMERICAN,COMPACT,FORD,3913.0,5054.0,7723.0,8707.0,3247.0,4384.0,6739.0,7911.0,NaN,NaN,19638,33619,FL,4000.0,0,1020,2009,12,50,7,0,341,False,False,False,False,False,False,1260144000


In [16]:
?train_cats

In [26]:
train_cats(df_raw)

In [28]:
df_raw.Size.cat.categories

Index(['COMPACT', 'CROSSOVER', 'LARGE', 'LARGE SUV', 'LARGE TRUCK', 'MEDIUM',
       'MEDIUM SUV', 'SMALL SUV', 'SMALL TRUCK', 'SPECIALTY', 'SPORTS', 'VAN'],
      dtype='object')

In [29]:
df_raw.Color.cat.categories

Index(['BEIGE', 'BLACK', 'BLUE', 'BROWN', 'GOLD', 'GREEN', 'GREY', 'MAROON',
       'NOT AVAIL', 'ORANGE', 'OTHER', 'PURPLE', 'RED', 'SILVER', 'WHITE',
       'YELLOW'],
      dtype='object')

In [30]:
df_raw.Color.cat.codes

0        12
1        14
2         7
3        13
4        13
         ..
72978     1
72979    13
72980    13
72981    14
72982    13
Length: 72983, dtype: int8

In [31]:
display_all(df_raw.isnull().sum().sort_index()/len(df_raw))

AUCGUART                             0.953153
Auction                              0.000000
BYRNO                                0.000000
Color                                0.000110
IsBadBuy                             0.000000
IsOnlineSale                         0.000000
MMRAcquisitionAuctionAveragePrice    0.000247
MMRAcquisitionAuctionCleanPrice      0.000247
MMRAcquisitionRetailAveragePrice     0.000247
MMRAcquisitonRetailCleanPrice        0.000247
MMRCurrentAuctionAveragePrice        0.004316
MMRCurrentAuctionCleanPrice          0.004316
MMRCurrentRetailAveragePrice         0.004316
MMRCurrentRetailCleanPrice           0.004316
Make                                 0.000000
Model                                0.000000
Nationality                          0.000069
PRIMEUNIT                            0.953153
PurchDay                             0.000000
PurchDayofweek                       0.000000
PurchDayofyear                       0.000000
PurchElapsed                      

In [32]:
os.makedirs('tmp', exist_ok=True)
df_raw.to_feather('tmp/cars-raw')

In [30]:
df_raw = pd.read_feather('tmp/cars-raw')

In [33]:
?proc_df

In [35]:
df, y, nas = proc_df(df_raw, 'IsBadBuy')

In [36]:
df.columns

Index(['RefId', 'Auction', 'VehYear', 'VehicleAge', 'Make', 'Model', 'Trim',
       'SubModel', 'Color', 'Transmission', 'WheelTypeID', 'WheelType',
       'VehOdo', 'Nationality', 'Size', 'TopThreeAmericanName',
       'MMRAcquisitionAuctionAveragePrice', 'MMRAcquisitionAuctionCleanPrice',
       'MMRAcquisitionRetailAveragePrice', 'MMRAcquisitonRetailCleanPrice',
       'MMRCurrentAuctionAveragePrice', 'MMRCurrentAuctionCleanPrice',
       'MMRCurrentRetailAveragePrice', 'MMRCurrentRetailCleanPrice',
       'PRIMEUNIT', 'AUCGUART', 'BYRNO', 'VNZIP1', 'VNST', 'VehBCost',
       'IsOnlineSale', 'WarrantyCost', 'PurchYear', 'PurchMonth', 'PurchWeek',
       'PurchDay', 'PurchDayofweek', 'PurchDayofyear', 'PurchIs_month_end',
       'PurchIs_month_start', 'PurchIs_quarter_end', 'PurchIs_quarter_start',
       'PurchIs_year_end', 'PurchIs_year_start', 'PurchElapsed',
       'WheelTypeID_na', 'MMRAcquisitionAuctionAveragePrice_na',
       'MMRAcquisitionAuctionCleanPrice_na',
       'MMRAc

In [39]:
df.head()

,RefId,Auction,VehYear,VehicleAge,Make,Model,Trim,SubModel,Color,Transmission,WheelTypeID,WheelType,VehOdo,Nationality,Size,TopThreeAmericanName,MMRAcquisitionAuctionAveragePrice,MMRAcquisitionAuctionCleanPrice,MMRAcquisitionRetailAveragePrice,MMRAcquisitonRetailCleanPrice,MMRCurrentAuctionAveragePrice,MMRCurrentAuctionCleanPrice,MMRCurrentRetailAveragePrice,MMRCurrentRetailCleanPrice,PRIMEUNIT,AUCGUART,BYRNO,VNZIP1,VNST,VehBCost,IsOnlineSale,WarrantyCost,PurchYear,PurchMonth,PurchWeek,PurchDay,PurchDayofweek,PurchDayofyear,PurchIs_month_end,PurchIs_month_start,PurchIs_quarter_end,PurchIs_quarter_start,PurchIs_year_end,PurchIs_year_start,PurchElapsed,WheelTypeID_na,MMRAcquisitionAuctionAveragePrice_na,MMRAcquisitionAuctionCleanPrice_na,MMRAcquisitionRetailAveragePrice_na,MMRAcquisitonRetailCleanPrice_na,MMRCurrentAuctionAveragePrice_na,MMRCurrentAuctionCleanPrice_na,MMRCurrentRetailAveragePrice_na,MMRCurrentRetailCleanPrice_na
0,1,1,2006,3,18,587,133,222,13,1,1.0,1,89046,3,6,4,8155.0,9829.0,11636.0,13600.0,7451.0,8552.0,11597.0,12409.0,0,0,21973,33619,6,7100.0,0,1113,2009,12,50,7,0,341,False,False,False,False,False,False,1260144000,False,False,False,False,False,False,False,False,False
1,2,1,2004,5,6,1,94,766,15,1,1.0,1,93593,1,5,1,6854.0,8383.0,10897.0,12572.0,7456.0,9222.0,11374.0,12791.0,0,0,19638,33619,6,7600.0,0,1053,2009,12,50,7,0,341,False,False,False,False,False,False,1260144000,False,False,False,False,False,False,False,False,False
2,3,1,2005,4,6,884,99,293,8,1,2.0,2,73807,1,6,1,3202.0,4760.0,6943.0,8457.0,4035.0,5557.0,7146.0,8702.0,0,0,19638,33619,6,4900.0,0,1389,2009,12,50,7,0,341,False,False,False,False,False,False,1260144000,False,False,False,False,False,False,False,False,False
3,4,1,2004,5,6,663,99,153,14,1,1.0,1,65617,1,1,1,1893.0,2675.0,4658.0,5690.0,1844.0,2646.0,4375.0,5518.0,0,0,19638,33619,6,4100.0,0,630,2009,12,50,7,0,341,False,False,False,False,False,False,1260144000,False,False,False,False,False,False,False,False,False
4,5,1,2005,4,7,369,127,53,14,2,2.0,2,69367,1,1,2,3913.0,5054.0,7723.0,8707.0,3247.0,4384.0,6739.0,7911.0,0,0,19638,33619,6,4000.0,0,1020,2009,12,50,7,0,341,False,False,False,False,False,False,1260144000,False,False,False,False,False,False,False,False,False


In [40]:
m = RandomForestRegressor(n_jobs=-1)
m.fit(df, y)
m.score(df,y)

0.8905522208200365

In [119]:
def split_vals(a,n): return a[:n].copy(), a[n:].copy()

n_valid = 12000  # same as Kaggle's test set size
n_trn = len(df)-n_valid
raw_train, raw_valid = split_vals(df_raw, n_trn)
X_train, X_valid = split_vals(df, n_trn)
y_train, y_valid = split_vals(y, n_trn)

X_train.shape, y_train.shape, X_valid.shape

((60983, 54), (60983,), (12000, 54))

In [120]:
m.score(X_valid, y_valid)

0.9090739647365292

In [121]:
?m.predict

In [174]:
m.predict(df)

array([0.07, 0.03, 0.12, ..., 0.01, 0.05, 0.06])

In [176]:
predictions = m.predict(df)

In [190]:
predictions.sort()
F_x = 0
tot_area = 0
total = sum(predictions)
for n in predictions:
    f_x = n
    F_x = F_x + f_x
    perc_0 = (F_x - f_x)/total
    perc_1 = F_x/total
    area = (perc_0 + perc_1 / 2)*(1/len(predictions))
    tot_area = tot_area + area
gini_coeff = tot_area
print(gini_coeff)    

0.2016141151572673


In [191]:
avg_kick_rate = sum(df_raw.IsBadBuy)/len(df_raw.IsBadBuy)
print(avg_kick_rate)
gini_coeff = gini_coeff/(1-avg_kick_rate)
print("The Gini coefficient based on random forest regressor is " + str(gini_coeff)) 
#233/570 in DontGetKicked Kaggle competition

0.12298754504473644
The Gini coefficient based on random forest regressor is 0.22988740241727995
